<a href="https://colab.research.google.com/github/hemil19/nlp-projects/blob/master/MachineHack%20Fake%20News%20Content%20Detection/Fake_News_MachineHack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn.metrics import *

!pip install catboost
from catboost import CatBoostClassifier

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 66.1MB 49kB/s 


In [ ]:
import nltk
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 71kB 3.3MB/s 
     |████████████████████████████████| 890kB 13.2MB/s 
     |████████████████████████████████| 1.1MB 53.4MB/s 
     |████████████████████████████████| 890kB 59.9MB/s 
     |████████████████████████████████| 3.0MB 47.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.6-cp36-none-any.whl size=101182 sha256=c026a0da725582c2247f5aa8d70c92272fd95ff4bfa874a894790c239041ea78
  Stored in directory: /root/.cache/pip/wheels/6f/3f/75/c0c4b3ef5dfbf8806d37b8dc661861772aba2f7aa419c85a9b
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=df4047be362de9895d7b0b595b1d09792af981c2ba6b122d8aedff39f006911a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
train = pd.read_csv('/content/Train.csv')
test = pd.read_csv('/content/Test.csv')

In [ ]:
train.shape,test.shape

((10240, 3), (1267, 2))

In [ ]:
len(train['Text_Tag'].unique())

3828

In [ ]:
train.head()

,Labels,Text,Text_Tag
0,1,Says the Annies List political group supports ...,abortion
1,2,When did the decline of coal start? It started...,"energy,history,job-accomplishments"
2,3,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy
3,1,Health care reform legislation is likely to ma...,health-care
4,2,The economic turnaround started at the end of ...,"economy,jobs"


In [ ]:
train.isnull().sum()

Labels      0
Text        0
Text_Tag    2
dtype: int64

In [ ]:
full_df = pd.concat([train, test]).reset_index()

In [ ]:
full_df.drop('index',axis=1,inplace=True)

In [ ]:
full_df.head()

,Labels,Text,Text_Tag
0,1.0,Says the Annies List political group supports ...,abortion
1,2.0,When did the decline of coal start? It started...,"energy,history,job-accomplishments"
2,3.0,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy
3,1.0,Health care reform legislation is likely to ma...,health-care
4,2.0,The economic turnaround started at the end of ...,"economy,jobs"


In [ ]:
full_df['Text_Tag'].fillna(full_df['Text_Tag'].mode()[0],inplace=True)

In [ ]:
full_df.isnull().sum()

Labels      1267
Text           0
Text_Tag       0
dtype: int64

In [ ]:
full_df['Length'] = full_df['Text'].apply(len)

In [ ]:
full_df

,Labels,Text,Text_Tag,Length
0,1.0,Says the Annies List political group supports ...,abortion,82
1,2.0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",141
2,3.0,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,105
3,1.0,Health care reform legislation is likely to ma...,health-care,78
4,2.0,The economic turnaround started at the end of ...,"economy,jobs",54
...,...,...,...,...
11502,NaN,Says his budget provides the highest state fun...,education,82
11503,NaN,Ive been here almost every day.,"civil-rights,crime,criminal-justice",31
11504,NaN,"In the early 1980s, Sen. Edward Kennedy secret...","bipartisanship,congress,foreign-policy,history",147
11505,NaN,Says an EPA permit languished under Strickland...,"environment,government-efficiency",96


In [ ]:
word_embeddings = SentenceTransformer('roberta-large-nli-stsb-mean-tokens').encode(full_df.Text)

100%|██████████| 1.31G/1.31G [01:01<00:00, 21.2MB/s]


In [ ]:
word_embeddings = pd.DataFrame(data=word_embeddings)

In [ ]:
word_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-0.843116,0.354265,0.356840,0.419612,0.502844,0.265882,1.124244,-0.337540,-1.011598,0.389833,-1.143914,-1.498768,0.349014,0.630597,-0.308929,-1.211498,0.639382,-0.070071,0.197311,-0.694955,-0.748949,-0.985391,-1.381263,-0.965012,0.196263,-0.165668,-0.192805,-0.399925,-0.571578,-0.015549,0.400864,-0.732438,0.259237,0.038886,-1.680590,-0.752561,0.343458,-0.785572,-1.548423,-0.629427,...,-0.416050,-0.434245,-0.105926,0.207333,-0.963042,-0.041006,-0.155755,0.277734,0.030981,-0.246306,0.552735,-0.799286,-0.250765,-1.899151,-0.518160,0.620628,0.464232,-0.309857,0.085775,0.445845,0.001797,0.197887,1.256549,-0.412362,-0.562702,-1.153979,0.355673,-0.240783,-1.033722,1.400331,0.022141,0.510810,0.853409,0.351011,0.043097,0.612750,-0.362310,0.728116,-0.631632,-0.265538
1,-0.154687,0.872684,1.772495,-0.295068,0.222306,-0.253727,-1.300732,0.190506,0.343303,-0.872295,0.883556,-0.409845,-0.357492,-0.139658,0.384984,-0.794848,-1.066066,-0.051637,0.139329,0.139747,0.117898,0.461542,-1.046781,0.199066,-1.169694,-0.055777,-0.828225,-1.178738,0.154195,-1.155196,0.319824,-0.137026,-0.893602,-0.493400,0.772719,-0.778965,-0.642557,-0.128266,1.243242,-1.194340,...,0.877192,0.133658,-0.019474,-0.440080,0.622852,-0.382756,-0.269156,-0.771481,-0.626759,-1.214526,1.213234,-0.706025,-0.896191,-0.587145,0.188101,0.173348,-1.696494,-0.575991,-0.401824,-0.878118,0.333844,-0.645788,-0.933317,0.095632,-1.189361,-0.747387,-0.634917,1.291250,-1.595213,0.180150,-0.493620,0.729953,-0.513199,1.092360,-0.258280,0.267085,-0.399993,-0.463843,-1.466223,0.220742
2,-1.090438,-1.039242,-0.442930,-1.469321,0.165641,-0.411884,-0.524986,1.435754,-0.221386,0.336803,-0.551852,-0.579336,-1.103296,1.014057,0.514499,0.020395,1.099065,1.208100,0.988841,1.010967,-0.034148,-1.322205,-1.929257,-0.313307,-0.597577,0.154905,0.451023,-0.220745,-0.292569,-0.569622,-1.047030,-1.128315,0.382468,1.066755,-0.906427,-0.978916,0.257975,-0.024067,-1.304761,0.307851,...,0.383583,-0.643160,-0.567529,0.672795,0.460018,1.943321,-0.099922,-0.545651,0.080034,-0.051012,0.285561,0.600909,-0.681902,1.709387,-0.631731,0.168583,-0.115426,0.145797,-1.153045,0.255024,-1.061559,0.622468,0.076881,-1.268890,0.564816,0.068601,-0.259890,0.401972,-0.329563,1.528327,0.896936,1.188237,-0.614642,0.053428,-0.562576,0.325781,-1.103370,-1.954508,0.481244,-0.146409
3,0.599439,0.623602,0.633820,0.405360,0.706751,-0.079209,0.344766,-0.220472,-0.569685,-0.079065,-0.076759,1.569410,-0.278343,0.464309,-0.433065,-1.224823,-0.210659,-0.190352,-0.811026,-1.186908,-0.193129,0.878079,1.618984,-0.496415,1.310309,-0.229878,-0.483368,-0.503116,0.748485,-0.606730,-1.408140,-0.482081,-0.711369,0.238350,-0.895125,0.667425,-0.624265,-0.758627,-0.358397,-0.065462,...,-0.312905,-0.635221,-0.146239,-0.417505,-0.173060,0.587924,0.703626,0.999840,-0.951556,-0.135231,0.494263,-0.434182,-1.080806,-0.713855,-0.633625,0.060905,0.127756,-0.921143,-0.846211,-0.678193,-0.489323,-1.591885,0.925409,-0.127750,0.404706,-0.285968,0.945851,-0.417140,-1.571513,1.861140,0.311481,0.002133,-0.285264,0.073709,0.787636,-0.344154,-0.042825,1.363382,0.595174,-0.336082
4,-0.092456,-0.040649,-0.215230,0.439570,1.114174,0.060738,-0.115374,-0.424731,0.706268,-0.155484,0.662839,0.791876,-0.067190,-0.313515,-0.166140,0.178111,-0.376365,0.253623,-1.166783,-1.275674,0.633949,0.408042,-1.205060,-1.429486,-0.148765,0.656819,-1.473106,0.556760,-0.075808,-1.473760,0.632215,1.001486,0.441519,0.425402,-0.661800,2.639184,0.492242,0.714776,0.791692,-0.303791,...,0.680720,1.496218,-0.320826,0.126426,0.195708,0.626634,-0.573824,0.262326,-0.785340,-0.277521,2.571323,1.039394,-0.520890,-1.399412,1.457771,0.471627,-2.558429,-0.643707,-0.414140,0.643783,0.277404,-2.057265,-0.369222,0.34327

In [ ]:
from sklearn.decomposition import PCA
pca = pd.DataFrame(data=PCA(10).fit_transform(word_embeddings), columns=[f'pca{i}' for i in range(1,11)])

In [ ]:
full_df = pd.concat([full_df.drop(['Text'], axis=1), word_embeddings, pca], axis=1)
full_df

,Labels,Text_Tag,Length,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10
0,1.0,abortion,82,-0.843116,0.354265,0.356840,0.419612,0.502844,0.265882,1.124244,-0.337540,-1.011598,0.389833,-1.143914,-1.498768,0.349014,0.630597,-0.308929,-1.211498,0.639382,-0.070071,0.197311,-0.694955,-0.748949,-0.985391,-1.381263,-0.965012,0.196263,-0.165668,-0.192805,-0.399925,-0.571578,-0.015549,0.400864,-0.732438,0.259237,0.038886,-1.680590,-0.752561,0.343458,...,0.552735,-0.799286,-0.250765,-1.899151,-0.518160,0.620628,0.464232,-0.309857,0.085775,0.445845,0.001797,0.197887,1.256549,-0.412362,-0.562702,-1.153979,0.355673,-0.240783,-1.033722,1.400331,0.022141,0.510810,0.853409,0.351011,0.043097,0.612750,-0.362310,0.728116,-0.631632,-0.265538,1.828868,-2.131068,1.054065,-4.184619,-0.503260,-7.653482,-3.988178,-4.354409,-1.857192,-1.943179
1,2.0,"energy,history,job-accomplishments",141,-0.154687,0.872684,1.772495,-0.295068,0.222306,-0.253727,-1.300732,0.190506,0.343303,-0.872295,0.883556,-0.409845,-0.357492,-0.139658,0.384984,-0.794848,-1.066066,-0.051637,0.139329,0.139747,0.117898,0.461542,-1.046781,0.199066,-1.169694,-0.055777,-0.828225,-1.178738,0.154195,-1.155196,0.319824,-0.137026,-0.893602,-0.493400,0.772719,-0.778965,-0.642557,...,1.213234,-0.706025,-0.896191,-0.587145,0.188101,0.173348,-1.696494,-0.575991,-0.401824,-0.878118,0.333844,-0.645788,-0.933317,0.095632,-1.189361,-0.747387,-0.634917,1.291250,-1.595213,0.180150,-0.493620,0.729953,-0.513199,1.092360,-0.258280,0.267085,-0.399993,-0.463843,-1.466223,0.220742,4.032442,0.974744,-0.600097,-6.140334,2.775310,5.042933,-2.242136,-0.386946,-1.737097,-4.009511
2,3.0,foreign-policy,105,-1.090438,-1.039242,-0.442930,-1.469321,0.165641,-0.411884,-0.524986,1.435754,-0.221386,0.336803,-0.551852,-0.579336,-1.103296,1.014057,0.514499,0.020395,1.099065,1.208100,0.988841,1.010967,-0.034148,-1.322205,-1.929257,-0.313307,-0.597577,0.154905,0.451023,-0.220745,-0.292569,-0.569622,-1.047030,-1.128315,0.382468,1.066755,-0.906427,-0.978916,0.257975,...,0.285561,0.600909,-0.681902,1.709387,-0.631731,0.168583,-0.115426,0.145797,-1.153045,0.255024,-1.061559,0.622468,0.076881,-1.268890,0.564816,0.068601,-0.259890,0.401972,-0.329563,1.528327,0.896936,1.188237,-0.614642,0.053428,-0.562576,0.325781,-1.103370,-1.954508,0.481244,-0.146409,11.340506,-4.129840,6.468379,-4.627543,0.989341,2.864509,2.183787,-2.280602,-1.413427,-2.472555
3,1.0,health-care,78,0.599439,0.623602,0.633820,0.405360,0.706751,-0.079209,0.344766,-0.220472,-0.569685,-0.079065,-0.076759,1.569410,-0.278343,0.464309,-0.433065,-1.224823,-0.210659,-0.190352,-0.811026,-1.186908,-0.193129,0.878079,1.618984,-0.496415,1.310309,-0.229878,-0.483368,-0.503116,0.748485,-0.606730,-1.408140,-0.482081,-0.711369,0.238350,-0.895125,0.667425,-0.624265,...,0.494263,-0.434182,-1.080806,-0.713855,-0.633625,0.060905,0.127756,-0.921143,-0.846211,-0.678193,-0.489323,-1.591885,0.925409,-0.127750,0.404706,-0.285968,0.945851,-0.417140,-1.571513,1.861140,0.311481,0.002133,-0.285264,0.073709,0.787636,-0.344154,-0.042825,1.363382,0.595174,-0.336082,-0.317844,-2.836286,-1.265430,2.281990,7.029100,-6.462660,-8.548338,-0.778524,-6.237104,0.158426
4,2.0,"economy,jobs",54,-0.092456,-0.040649,-0.215230,0.439570,1.114174,0.060738,-0.115374,-0.424731,0.706268,-0.155484,0.662839,0.791876,-0.067190,-0.313515,-0.166140,0.178111,-0.376365,0.253623,-1.166783,-1.275674,0.633949,0.408042,-1.205060,-1.429486,-0.148765,0.656819,-1.473106,0.556760,-0.075808,-1.473760,0.632215,1.001486,0.441519,0.425402,-0.661800,2.639184,0.492242,...,2.571323,1.039394,-0.520890,-1.399412,1.457771,0.471627,-2.558429,-0.643707,-0.414140,0.643783,0.277404,-2.057265,-0.369222,0.343270,-0.080974,-1.999635,0.116802,-0.130946,-2.222349,0.629590,-0.951390,-0.129189,-0.908807

In [ ]:
train_df = full_df[full_df.Labels.notnull()]
test_df = full_df[full_df.Labels.isnull()]

In [ ]:
train_df.isnull().sum()

Labels      0
Text_Tag    0
Length      0
0           0
1           0
           ..
pca6        0
pca7        0
pca8        0
pca9        0
pca10       0
Length: 1037, dtype: int64

In [ ]:
test_df.isnull().sum()

Labels      1267
Text_Tag       0
Length         0
0              0
1              0
            ... 
pca6           0
pca7           0
pca8           0
pca9           0
pca10          0
Length: 1037, dtype: int64

In [ ]:
X, y = train_df.drop(['Labels'], axis=1), train_df.Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
X.head()

,Text_Tag,Length,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10
0,abortion,82,-0.843116,0.354265,0.356840,0.419612,0.502844,0.265882,1.124244,-0.337540,-1.011598,0.389833,-1.143914,-1.498768,0.349014,0.630597,-0.308929,-1.211498,0.639382,-0.070071,0.197311,-0.694955,-0.748949,-0.985391,-1.381263,-0.965012,0.196263,-0.165668,-0.192805,-0.399925,-0.571578,-0.015549,0.400864,-0.732438,0.259237,0.038886,-1.680590,-0.752561,0.343458,-0.785572,...,0.552735,-0.799286,-0.250765,-1.899151,-0.518160,0.620628,0.464232,-0.309857,0.085775,0.445845,0.001797,0.197887,1.256549,-0.412362,-0.562702,-1.153979,0.355673,-0.240783,-1.033722,1.400331,0.022141,0.510810,0.853409,0.351011,0.043097,0.612750,-0.362310,0.728116,-0.631632,-0.265538,1.828868,-2.131068,1.054065,-4.184619,-0.503260,-7.653482,-3.988178,-4.354409,-1.857192,-1.943179
1,"energy,history,job-accomplishments",141,-0.154687,0.872684,1.772495,-0.295068,0.222306,-0.253727,-1.300732,0.190506,0.343303,-0.872295,0.883556,-0.409845,-0.357492,-0.139658,0.384984,-0.794848,-1.066066,-0.051637,0.139329,0.139747,0.117898,0.461542,-1.046781,0.199066,-1.169694,-0.055777,-0.828225,-1.178738,0.154195,-1.155196,0.319824,-0.137026,-0.893602,-0.493400,0.772719,-0.778965,-0.642557,-0.128266,...,1.213234,-0.706025,-0.896191,-0.587145,0.188101,0.173348,-1.696494,-0.575991,-0.401824,-0.878118,0.333844,-0.645788,-0.933317,0.095632,-1.189361,-0.747387,-0.634917,1.291250,-1.595213,0.180150,-0.493620,0.729953,-0.513199,1.092360,-0.258280,0.267085,-0.399993,-0.463843,-1.466223,0.220742,4.032442,0.974744,-0.600097,-6.140334,2.775310,5.042933,-2.242136,-0.386946,-1.737097,-4.009511
2,foreign-policy,105,-1.090438,-1.039242,-0.442930,-1.469321,0.165641,-0.411884,-0.524986,1.435754,-0.221386,0.336803,-0.551852,-0.579336,-1.103296,1.014057,0.514499,0.020395,1.099065,1.208100,0.988841,1.010967,-0.034148,-1.322205,-1.929257,-0.313307,-0.597577,0.154905,0.451023,-0.220745,-0.292569,-0.569622,-1.047030,-1.128315,0.382468,1.066755,-0.906427,-0.978916,0.257975,-0.024067,...,0.285561,0.600909,-0.681902,1.709387,-0.631731,0.168583,-0.115426,0.145797,-1.153045,0.255024,-1.061559,0.622468,0.076881,-1.268890,0.564816,0.068601,-0.259890,0.401972,-0.329563,1.528327,0.896936,1.188237,-0.614642,0.053428,-0.562576,0.325781,-1.103370,-1.954508,0.481244,-0.146409,11.340506,-4.129840,6.468379,-4.627543,0.989341,2.864509,2.183787,-2.280602,-1.413427,-2.472555
3,health-care,78,0.599439,0.623602,0.633820,0.405360,0.706751,-0.079209,0.344766,-0.220472,-0.569685,-0.079065,-0.076759,1.569410,-0.278343,0.464309,-0.433065,-1.224823,-0.210659,-0.190352,-0.811026,-1.186908,-0.193129,0.878079,1.618984,-0.496415,1.310309,-0.229878,-0.483368,-0.503116,0.748485,-0.606730,-1.408140,-0.482081,-0.711369,0.238350,-0.895125,0.667425,-0.624265,-0.758627,...,0.494263,-0.434182,-1.080806,-0.713855,-0.633625,0.060905,0.127756,-0.921143,-0.846211,-0.678193,-0.489323,-1.591885,0.925409,-0.127750,0.404706,-0.285968,0.945851,-0.417140,-1.571513,1.861140,0.311481,0.002133,-0.285264,0.073709,0.787636,-0.344154,-0.042825,1.363382,0.595174,-0.336082,-0.317844,-2.836286,-1.265430,2.281990,7.029100,-6.462660,-8.548338,-0.778524,-6.237104,0.158426
4,"economy,jobs",54,-0.092456,-0.040649,-0.215230,0.439570,1.114174,0.060738,-0.115374,-0.424731,0.706268,-0.155484,0.662839,0.791876,-0.067190,-0.313515,-0.166140,0.178111,-0.376365,0.253623,-1.166783,-1.275674,0.633949,0.408042,-1.205060,-1.429486,-0.148765,0.656819,-1.473106,0.556760,-0.075808,-1.473760,0.632215,1.001486,0.441519,0.425402,-0.661800,2.639184,0.492242,0.714776,...,2.571323,1.039394,-0.520890,-1.399412,1.457771,0.471627,-2.558429,-0.643707,-0.414140,0.643783,0.277404,-2.057265,-0.369222,0.343270,-0.080974,-1.999635,0.116802,-0.130946,-2.222349,0.629590,-0.9

In [ ]:
X_train.head()

,Text_Tag,Length,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10
550,"energy,environment,gas-prices,trade",56,-0.899627,-0.882568,-0.880872,1.354892,0.059783,-0.324811,-0.085154,0.599406,0.170106,1.042004,0.447218,2.177853,0.017705,-0.897432,0.005607,-0.004105,-1.841052,0.251485,-0.374242,-0.080755,0.733789,-0.715286,-1.042610,-0.914718,1.354150,0.071085,0.240315,0.041842,-0.306044,-2.211933,0.192219,0.479538,-0.497643,-0.236034,0.473266,0.079551,-0.469660,0.068286,...,1.620116,0.211448,-0.606983,-0.837320,0.884512,-0.654121,-0.835717,-0.081681,0.712548,-1.163509,0.359526,-1.424104,-0.568836,0.062424,0.105435,-1.118217,-1.146313,0.091596,-0.463212,1.248872,-0.331523,0.881444,-0.602026,-0.322495,-0.750720,-0.178657,-0.563850,1.576189,-0.469349,0.630362,0.976791,7.013562,-1.668056,2.724199,7.478570,0.158301,2.281113,0.603589,-1.805534,-9.424116
7231,"foreign-policy,military,abc-news-week",51,-1.104368,-0.409978,-0.669678,-0.422685,0.260004,1.185878,-0.408552,0.640778,-1.476153,-0.180429,0.014837,0.295234,0.822766,-0.739990,1.794060,2.152598,-0.060641,-0.535848,0.586863,1.804089,1.477044,-0.173402,-0.404568,-1.148337,1.284695,-0.286499,-0.897991,-0.454339,-0.389162,-1.027658,0.729403,-0.559129,-0.028135,-0.658531,-0.760322,-1.526029,-0.364384,-0.017957,...,1.038084,-0.015240,0.848897,0.159687,1.514579,-0.138271,0.426749,-2.251547,0.348714,-1.597943,-1.427150,-0.167259,-0.884988,0.462680,-0.426133,-0.236224,-0.287911,-0.632071,-0.604559,0.534084,0.657045,0.982440,-0.137405,-1.169338,-0.659197,1.016031,-0.109025,-0.533322,-1.739792,-0.752737,8.777389,-4.479741,-7.936833,2.467112,0.184897,3.668072,0.895390,5.385603,1.270239,-4.572798
2515,"criminal-justice,jobs",166,-0.755584,0.646944,0.417901,0.138028,0.796935,0.541305,-0.216255,0.042771,0.353116,0.517065,-2.159441,-1.873956,0.514674,0.279156,0.909459,-0.586071,0.924904,1.328719,-0.136519,1.447375,1.231256,0.765696,-0.012322,-0.157271,0.819364,0.462460,-0.964923,-0.148687,-0.233388,0.139507,0.659101,-0.508215,0.865243,-0.667399,-1.194827,0.156496,-0.800705,0.299527,...,0.471170,-1.139725,-0.596468,-0.396888,-0.714967,0.570163,-1.376891,-1.703961,0.147327,0.290006,-0.992880,0.020235,-1.015480,0.156960,0.287558,-0.207406,-0.341815,-0.064206,-0.862320,0.042637,0.503020,-0.333902,0.313677,0.520505,0.342270,0.771678,-0.749383,-1.635118,-0.782214,-0.277262,0.257617,3.420678,-5.555322,-7.239256,-7.108066,3.961646,-2.390929,4.395992,6.163784,-0.917058
4466,"climate-change,economy,environment,government-...",155,1.004852,1.675487,0.278537,0.560506,1.520011,-0.098680,-0.576232,0.756510,0.842200,0.355231,-0.036993,-0.798921,0.660707,1.004127,0.197737,-2.240191,-0.043043,1.127437,-0.043940,-0.910211,0.818543,0.298127,1.596169,0.766473,-0.005528,-0.705082,-3.069271,0.071130,1.052419,-0.022477,-0.593169,1.192770,0.205464,0.205373,-1.510930,0.470068,-0.865624,-0.690903,...,1.369049,0.659581,-0.783643,-2.157387,-0.284602,0.380962,-1.233035,-0.989608,-1.262704,-0.687679,-0.506396,0.486844,0.887867,-0.148420,0.175094,-0.341891,0.452805,0.966130,-1.213985,1.246686,-0.528331,-0.024969,-0.200753,1.551862,0.351165,2.012350,0.054932,0.190591,0.419582,-0.326326,-10.448917,-2.917379,-4.739170,2.244374,-1.536720,-9.395209,-1.847345,-3.545555,-0.421461,2.730966
211,"corrections-and-updates,federal-budget,health-...",58,0.105385,0.987429,-0.582443,-0.000981,0.911445,-0.352457,0.235195,0.417940,-0.102367,0.152098,0.400899,1.255271,0.019346,1.237568,0.134347,-0.584170,-1.084678,0.447441,0.264982,0.465877,0.627901,0.461317,-0.110676,-0.259780,1.035852,0.323600,-0.431699,-0.140644,0.669916,-0.773980,-1.305484,0.474953,-0.658416,-0.734810,0.187097,0.417155,-0.788675,0.253271,...,0.908152,1.264234,-0.368734,-2.698105,0.388268,0.927492,-0.504060,-1.064969,-1.043479,

In [ ]:
model_cat = CatBoostClassifier(od_type='Iter', iterations=10000, task_type='GPU')
model_cat.fit(X_train, y_train.astype(int),
              eval_set=(X_test, y_test.astype(int)),
              early_stopping_rounds=100,
              cat_features=['Text_Tag'])

Learning rate set to 0.047543
0:	learn: 1.7881567	test: 1.7888066	best: 1.7888066 (0)	total: 28.5ms	remaining: 4m 44s
1:	learn: 1.7827632	test: 1.7847500	best: 1.7847500 (1)	total: 59ms	remaining: 4m 54s
2:	learn: 1.7779415	test: 1.7813040	best: 1.7813040 (2)	total: 84.7ms	remaining: 4m 42s
3:	learn: 1.7729893	test: 1.7776752	best: 1.7776752 (3)	total: 111ms	remaining: 4m 36s
4:	learn: 1.7687640	test: 1.7747191	best: 1.7747191 (4)	total: 136ms	remaining: 4m 31s
5:	learn: 1.7643297	test: 1.7715740	best: 1.7715740 (5)	total: 162ms	remaining: 4m 30s
6:	learn: 1.7596916	test: 1.7682068	best: 1.7682068 (6)	total: 190ms	remaining: 4m 31s
7:	learn: 1.7560666	test: 1.7658441	best: 1.7658441 (7)	total: 214ms	remaining: 4m 27s
8:	learn: 1.7523057	test: 1.7636793	best: 1.7636793 (8)	total: 241ms	remaining: 4m 27s
9:	learn: 1.7492287	test: 1.7613621	best: 1.7613621 (9)	total: 265ms	remaining: 4m 25s
10:	learn: 1.7455369	test: 1.7587630	best: 1.7587630 (10)	total: 290ms	remaining: 4m 23s
11:	learn:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(model_cat.predict(X_test),y_test)#0.2563 ->0.2675(with 1st text tag feature) ->0.2734(with both text tag feature)

0.2568359375

In [ ]:
prediction = model_cat.predict_proba(test_df[X_train.columns])
submission = pd.DataFrame(prediction, columns=[0,1,2,3,4,5])
submission.to_csv('final_fake.csv', index=False)
submission#1.68256

,0,1,2,3,4,5
0,0.177288,0.173604,0.188109,0.205749,0.070155,0.185096
1,0.131122,0.216583,0.253889,0.222244,0.042681,0.133482
2,0.134616,0.267826,0.184572,0.129146,0.085332,0.198508
3,0.315550,0.183705,0.196007,0.097573,0.119260,0.087904
4,0.121498,0.286285,0.187049,0.147281,0.166020,0.091867
...,...,...,...,...,...,...
1262,0.121385,0.123693,0.292726,0.272475,0.025050,0.164671
1263,0.160795,0.210239,0.138422,0.159379,0.057151,0.274014
1264,0.188865,0.189383,0.190857,0.114794,0.160239,0.155863
1265,0.130486,0.155202,0.279535,0.220419,0.071255,0.143104
